In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

2017-04-21 20:53:03,380 : INFO : Starting new HTTPS connection (1): api.twitter.com


Switching to API Credentials #1


In [2]:
s = open_session()
all_tweets_es = [t for t in s.query(Tweet).all() if t.lang == 'es']
all_tweets_text_es = [t.text for t in all_tweets_es]

# with open('all_tweets_text_es.json', 'w') as f:
#     json.dump(all_tweets_text_es, f)

In [3]:
all_tweets_es_ids = [t.id for t in all_tweets_es]

In [4]:
len(all_tweets_text_es)    

109040

In [5]:
tweets = all_tweets_text_es

# OK, let's finally do some topic modelling

In [6]:
dictionary = gensim.corpora.Dictionary.load("tweets_es_bigvocab.dict")

2017-04-21 20:53:33,373 : INFO : loading Dictionary object from tweets_es_bigvocab.dict


In [7]:
with open('tweets_es_bow_bigvocab.pickle', 'rb') as f:
    bow = pickle.load(f)

In [8]:
from math import ceil

In [ ]:
n_topics = 15
iters = 100
passes = 10
workers = 8
chunksize = int(ceil(len(bow)*1.0/workers))

In [ ]:
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=chunksize,
        workers=workers
)
model.save("tweets_es_%dtopics.lda" % n_topics)

2017-04-21 20:53:56,621 : INFO : using serial LDA version on this node
2017-04-21 20:54:02,665 : INFO : running online LDA training, 15 topics, 10 passes over the supplied corpus of 109040 documents, updating every 109040 documents, evaluating every ~109040 documents, iterating 100x with a convergence threshold of 0.001000
2017-04-21 20:54:02,669 : INFO : training LDA model using 8 processes
2017-04-21 20:54:03,254 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #13630/109040, outstanding queue size 1
2017-04-21 20:54:03,750 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #27260/109040, outstanding queue size 2
2017-04-21 20:54:04,189 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #40890/109040, outstanding queue size 3
2017-04-21 20:54:04,712 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #54520/109040, outstanding queue size 4
2017-04-21 20:54:04,977 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to 

In [18]:
model = gensim.models.LdaModel.load("tweets_es_%dtopics.lda" % n_topics)

2017-04-21 19:58:38,615 : INFO : loading LdaModel object from tweets_es_20topics.lda
2017-04-21 19:58:38,660 : INFO : setting ignored attribute state to None
2017-04-21 19:58:38,661 : INFO : setting ignored attribute dispatcher to None
2017-04-21 19:58:38,663 : INFO : loading LdaModel object from tweets_es_20topics.lda.state


In [ ]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("tweets_es_%d.viz" % n_topics,'wb'))

In [20]:
viz = pickle.load(open("tweets_es_%d.viz" % n_topics,'rb'))

In [ ]:
pyLDAvis.display(viz)

# Extracción de features LDA sobre tweets

In [ ]:
tweets_lda = [model[t_bow] for t_bow in bow]

In [ ]:
def rows_to_csc(rows):
    data = []
    row_ind = []
    col_ind = []
    for i, r in enumerate(rows):
        for j, d in r:
            row_ind.append(i)
            col_ind.append(j)
            data.append(d)
    return csc_matrix((data, (row_ind, col_ind)))

In [ ]:
from scipy.sparse import csc, csc_matrix

In [ ]:
X_tweets_lda = rows_to_csc(tweets_lda)

In [ ]:
X_tweets_lda_df = pd.DataFrame(index=all_tweets_es_ids, data=X_tweets_lda.todense())

In [ ]:
X_tweets_lda_df

In [ ]:
X_tweets_lda_df.to_pickle("alltweets_es_lda%d.pickle" % n_topics)